# Running a VASP job in pyiron

Important: This assumes that you will have configured VASP already - this notebook _doesn't_ work out of the box

If you haven't, configure here:

https://pyiron.readthedocs.io/en/latest/source/installation.html

In [1]:
from pyiron.project import Project

In [2]:
pr = Project("run_vasp_calc")

In [3]:
structure = pr.create.structure.ase.bulk("Fe",crystalstructure="bcc", cubic=True)*[2,2,2]

In [4]:
print(len(structure))

16


In [5]:
job = pr.create_job(job_type=pr.job_type.Vasp, job_name = "bcc_Fe_16_atoms")

# Set the structure
job.structure = structure

# Set gamma-centered mesh of 2, 2, 2 subdivisions
job.set_kpoints(mesh=[2, 2, 2], scheme = "GC")

# Set/adjust incar flags
job.input.incar['EDIFF'] = 1E-7
job.input.incar['EDIFFG'] = -1E-2

# Set your queue that you already configured in installation (queue.yaml)
job.server.queue = "cmti"
job.server.cores = 40
job.executable = "5.4.4_mpi"

In [6]:
job.run(run_mode="queue", delete_existing_job=True)

The job bcc_Fe_16_atoms was saved and received the ID: 20407761
Queue system id:  5874931


In [7]:
pr.job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,20407761,submitted,Fe16,bcc_Fe_16_atoms,/bcc_Fe_16_atoms,/cmmc/u/,hmai/MaterialDigital/run_vasp_calc/,2023-07-31 12:59:07.119963,None,None,hmai@cmti001#40#cmti,Vasp,5.4.4_mpi,None,None


In [8]:
job.output.energy_tot

array([-121.96485025])

## Accessing job output

If we want to look at the raw output files, we can!

Access the stored output in the HDF5 file in "inspect" mode via calling 

job["YOUR_OUTPUT_FILE"]

In [9]:
job["OUTCAR"][:20]

[' vasp.5.4.4.18Apr17-6-g9f103f2a35 (build Feb 19 2020 16:12:31) complex          \n',
 '  \n',
 ' executed on             LinuxIFC date 2023.07.31  14:03:57\n',
 ' running on   40 total cores\n',
 ' distrk:  each k-point on   40 cores,    1 groups\n',
 ' distr:  one band on NCORES_PER_BAND=   1 cores,   40 groups\n',
 '\n',
 '\n',
 '--------------------------------------------------------------------------------------------------------\n',
 '\n',
 '\n',
 ' INCAR:\n',
 ' POTCAR:    PAW_PBE Fe 06Sep2000                  \n',
 '\n',
 ' ----------------------------------------------------------------------------- \n',
 '|                                                                             |\n',
 '|           W    W    AA    RRRRR   N    N  II  N    N   GGGG   !!!           |\n',
 '|           W    W   A  A   R    R  NN   N  II  NN   N  G    G  !!!           |\n',
 '|           W    W  A    A  R    R  N N  N  II  N N  N  G       !!!           |\n',
 '|           W WW W  AAAAAA  RRR

We usually store the most important outputs in hdf5

Access and navigate the stored information using the same syntax

In [10]:
job['output']

{'groups': ['charge_density', 'electronic_structure', 'generic', 'outcar', 'structure'], 'nodes': ['description']}

In [11]:
job['output/generic/']

{'groups': ['dft'], 'nodes': ['cells', 'elastic_constants', 'energy_pot', 'energy_tot', 'forces', 'positions', 'pressures', 'steps', 'stresses', 'temperature', 'volume']}

In [12]:
# Let's look at the energy_tot and forces
energy, forces = job['output/generic/energy_tot'], job['output/generic/forces']
energy, forces

(array([-121.96485025]),
 array([[[-0., -0.,  0.],
         [-0., -0., -0.],
         [ 0., -0., -0.],
         [-0., -0., -0.],
         [ 0.,  0.,  0.],
         [-0.,  0.,  0.],
         [ 0., -0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [-0.,  0., -0.],
         [-0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [-0., -0., -0.],
         [-0., -0.,  0.],
         [ 0., -0., -0.],
         [ 0., -0., -0.]]]))